# Gravitational Model as Class

This class enables to generate random configuration based on a gravitational model. The probability for a module to be part of the building or to be neglected depend on the proximity to one or more gravity point. Gravity Point can be more than one, can be 2 or 3-dimensional, can be set on the corners or on random location within the playfield, can have different weights.
The class export the list of perimeters with gaps as csv file

In [2]:
class GraviationalModel():
    
    def __init__(self, params):
        
        import math
        from itertools import repeat
        self.experiment = params["experiment"]
        self.folder = params["folder"]
        self.iters = params["iters"]
        self.shape = params["shape"]
        self.BL = params["shape"][0] # building length
        self.BW = params["shape"][1] # building width
        self.BH = params["shape"][2] # building height
        self.ML = params["shape"][3] # module length
        self.MW = params["shape"][4] # module width
        self.MH = params["shape"][5] # module height
        self.exceptions = params["exceptions"][:]
        self.MaxArea = params["MaximalArea"]
        self.GForce = params["GForce"]
        self.Weights = params["CornerWeights"]
        self.Corners = params["corners"]
        self.GCenters = params["gravityCenters"]
        self.dim = params["dim"]
        self.Cantilever = params["CantileverRule"]
        self.CompactBuilding = params["CompactBuildingRule"]
        self.RoomDepth = params["RoomDepthRule"]
        self.verbose = params["verbose"]
        self.modx = math.floor(self.BL / self.ML) # number of modules on X axis
        self.mody = math.floor(self.BW / self.MW) # number of modules on Y axis
        self.modz = math.floor(self.BH / self.MH) # number of modules on Z axis
        self.recs = [[[1 for x in repeat(None, self.modx)] for x in repeat(None, self.mody)] for x in repeat(None, self.modz)]
        self.mods_max = self.modx * self.mody * self.modz # maximal number of modules in the defined volume
        self.mods_goal = math.floor(self.MaxArea / (self.ML * self.MW)) # desired number of modules to achive MaxArea
        self.mods_free = [math.floor(21 / self.ML), math.floor(21 / self.MW)] # maximal number of modules to receive an adequate sunlight
        
    def get_center(self, obj):
        centers = []
        for level in range(len(obj)):
            Z = level * self.MH
            coordslevel = []
            for row in range(len(obj[level])):
                Y = row * self.MW + self.MW / 2 # center point of the rectangle
                coordsrow = []
                for rec in range(len(obj[level][row])):
                    X = rec * self.ML + self.ML / 2 #center point of the rectangle
                    # recs is the list of possible modules, therefore all the values are 1
                    # recsel is the selection of modules according to multiple rules, it has 1 and 0 
                    if obj[level][row][rec] == 1:
                        coordsrow.append([X, Y, Z])
                coordslevel.append(coordsrow)
            centers.append(coordslevel)
        return(centers)
    
    def distanceMatrix(self):
        
        import random
        import math
        from itertools import repeat
        
        # this function calculates the distance of each point to the corner points, given the following structure of coords
        # coords = [[[P111[x,y,z], P112[x,y,z], ..., P11N[x,y,z]], [P121[x,y,z], P122[x,y,z], ..., P12N[x, y, z]], ... ]
        # corners
        self.centers = self.get_center(self.recs)
        if self.Corners == True:
            gravityPoints = [self.centers[0][0][0], self.centers[0][0][-1], self.centers[0][-1][0], self.centers[0][-1][-1]]
        else:
            gravityPoints = []
            for g in range(self.GCenters):
                selx = random.sample(list(range(self.modx)), 1)
                sely = random.sample(list(range(self.mody)), 1)
                selz = random.sample(list(range(self.modz)), 1)
                gravityPoints.append(self.centers[selz[0]][sely[0]][selx[0]])
        # output
        self.dm = [[[[[] for x in repeat(None, self.GCenters)] for x in repeat(None, self.modx)] for x in repeat(None, self.mody)] for x in repeat(None, self.modz)]
        # main loop
        for level in range(len(self.centers)):
            for row in range(len(self.centers[level])):
                for el in range(len(self.centers[level][row])):
                    X0 = self.centers[level][row][el][0]
                    Y0 = self.centers[level][row][el][1]
                    Z0 = self.centers[level][row][el][2]
                    for c in range(len(gravityPoints)):
                        if self.dim == 2:
                            d = math.sqrt((X0 - gravityPoints[c][0]) ** 2 + (Y0 - gravityPoints[c][1]) ** 2)
                        if self.dim == 3:
                            d = math.sqrt((X0 - gravityPoints[c][0]) ** 2 + (Y0 - gravityPoints[c][1]) ** 2 + (Z0 - gravityPoints[c][2]) ** 2)
                        # print(d)
                        self.dm[level][row][el][c] = d
        return(self.dm)
    
    def gravitycoehfficients(self): # coords, gforce, weights, corners = True, gravityCenters = 4, dim = 2):
        from itertools import repeat
        import math
        
        # running distance matrix function
        self.distanceMatrix()
        
        self.gr = [[[[] for x in repeat(None, self.modx)] for x in repeat(None, self.mody)] for x in repeat(None, self.modz)]
        # dm = distanceMatrix(coords, corners = corners, gravityCenters = gravityCenters, dimensions = dim)
        for level in range(self.modz):
            for row in range(self.mody):
                for el in range(self.modx):
                    distances = self.dm[level][row][el]
                    gravity = 0
                    for d in range(len(distances)):
                        gravity += self.Weights[d] * math.exp(-self.GForce * distances[d])
                    self.gr[level][row][el] = gravity
        return (self.gr)
    
    def get_exceptions(self, obj, exceptions):
        
        import math
        
        res = list.copy(obj)
        
        for exception in exceptions:
            x = math.floor(exception[0] / self.ML) 
            y = math.floor(exception[1] / self.MW)
            z = math.floor(exception[2] / self.MH)
            l = math.ceil(exception[3] / self.ML)
            w = math.ceil(exception[4] / self.MW)
            h = math.ceil(exception[5] / self.MH)

            for level in range(len(res)):
                for row in range(len(res[level])):
                    for el in range(len(res[level][row])):
                        condx = el >= x and el < x + l
                        condy = row >= y and row < y + w
                        condz = level >= z and level < z + h
                        if (condx and condy and condz):
                            res[level][row][el] = 0
        return(res)
    
    def cantilever(self, obj):
        import math
        
        length = len(obj[0][0])
        width = len(obj[0])
        
        cantX = math.floor(self.Cantilever / self.ML) # cantilever modules in X direction
        cantY = math.floor(self.Cantilever / self.MW) # cantilever modules in Y direction
        
        for level in range(len(obj)):
            for row in range(width):
                for el in range(length):
                    if level > 0:
                        sostain = obj[level - 1][row][el]
                        if cantX > 0:
                            for step in range(1, cantX + 1):
                                if (el + step in list(range(length))):
                                    sostain += obj[level - 1][row][el + step]
                                if (el - step in list(range(length))):
                                    sostain += obj[level - 1][row][el - step]
                        if cantY > 0:
                            for w in range(1, cantY + 1):
                                if (row + w in list(range(width))):
                                    sostain += obj[level - 1][row + w][el]
                                if (row - w in list(range(width))):
                                    sostain += obj[level - 1][row - w][el]
                        if sostain == 0:
                            obj[level][row][el] = 0
        return(obj)
    
    def compact_building(self, obj):
        for level in range(len(obj)):
            for row in range(len(obj[level])):
                for el in range(len(obj[level][row])):
                    # avoiding the perimeter modules
                    if (el > 0 and el < len(obj[level][row]) - 1 and row > 0 and row < len(obj[level]) - 1):
                        cond1 = obj[level][row][el - 1] == 0
                        cond2 = obj[level][row][el + 1] == 0
                        cond3 = obj[level][row + 1][el] == 0
                        cond4 = obj[level][row - 1][el] == 0
                        if not (cond1 or cond2 or cond3 or cond4):
                            obj[level][row][el] = 1
        return(obj)
    
    def hardroomdepth(self, obj):
        for level in range(len(obj)):
            for row in range(len(obj[level])):
                # if the module is not close to a Y border
                if row >= self.mods_free[1] and row < len(obj[level]) - self.mods_free[1]:
                    for el in range(len(obj[level][row])):
                        #if a module is not close to a X border
                        if el >= self.mods_free[0] and el < len(obj[level][row]) - self.mods_free[0]:
                            # hard rule
                            obj[level][row][el] = 0
        return(obj)
    
    def softroomdepth(self, obj):
        for level in range(len(obj)):
            for row in range(len(obj[level])):
                # if the module is not close to a Y border
                if row >= self.mods_free[1] and row < len(obj[level]) - self.mods_free[1]:
                    for el in range(len(obj[level][row])):
                        #if a module is not close to a X border
                        if el >= self.mods_free[0] and el < len(obj[level][row]) - self.mods_free[0]:
                            # soft rule:
                            # no opening within n modules on x axes 
                            cond1 = sum(obj[level][row][el - self.mods_free[0] : el + self.mods_free[0] + 1]) == 2 * self.mods_free[0] + 1
                            # no opening within n modules on y axes
                            ymods = 0
                            for i in range(- self.mods_free[1], self.mods_free[1] + 1):
                                ymods += obj[level][row + i][el] 
                            cond2 = ymods == 2 * self.mods_free[1] + 1
                            if cond1 and cond2:
                                obj[level][row][el] = 0
        return(obj)
    
    def random_configuration(self):
        # Load packages
        import csv
        import datetime
        import random
        from itertools import repeat
        import math
        import numpy as np
        
        # OUTPUT
        self.combinations = [] # List containing all the possible combinations, main output
        # Check if it's possible to achive the desired area
        # theoretically possible modules after exceptions
        self.possible = [[[1 for x in repeat(None, self.modx)] for x in repeat(None, self.mody)] for x in repeat(None, self.modz)]
        # Applying exceptions
        if len(self.exceptions) > 0:
            self.possible = self.get_exceptions(self.possible, self.exceptions)
        # Applying the cantilever rule
        if self.Cantilever > 0:
            self.possible = self.cantilever(self.possible)
        # Applying Room depth
        if self.RoomDepth == "hard":
            self.possible = self.hardroomdepth(self.possible)
        if self.RoomDepth == "soft":
            self.possible = self.softroomdepth(self.possible)
        # breaking the process if the desired floor area can not be reached
        self.mods_poss = sum([item for sublist in [item for sublist in self.possible for item in sublist] for item in sublist])
        if self.mods_goal > self.mods_poss:
            return(print("Impossible to reach the desired area with the parameters given"))
        if self.verbose:
            print("Maximal number of modules according to the geometry and exceptions: ", self.mods_poss)
            print("Maximal modules according to construction law:", self.mods_goal)
            print("*" * 30)
        
        # coordinates of the module centers
        self.module_coords = self.get_center(self.recs)
        # main loop
        for i in range(self.iters):
            # gravity cohefficient
            self.gr = self.gravitycoehfficients()
            grflat = [item for sublist in [item for sublist in self.gr for item in sublist] for item in sublist]
            gr_std = np.std(grflat)
            # static target depend on the distribution of the gravity coehefficients and on the ratio
            # between desired an possible modules
            static_target = np.quantile(grflat, (1 - self.mods_goal / self.mods_poss)) + 2 * gr_std
            # plt.hist(grflat, bins = 50)
            recsel = [[[0 for x in repeat(None, self.modx)] for x in repeat(None, self.mody)] for x in repeat(None, self.modz)]
            flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
            self.area = sum(flatrecs) * self.MW * self.ML
            if self.verbose:
                print("Mean and SD of gravity cohefficient distribution: ", np.mean(grflat), gr_std)
                print("Elements over the static threshold: ", sum(grflat > static_target))
            
            # Shuffling the selectors in order to pick different modules
            zsel = list(range(len(self.recs)))
            random.shuffle(zsel)
            ysel = list(range(len(self.recs[0])))
            random.shuffle(ysel)
            xsel = list(range(len(self.recs[0][0])))
            random.shuffle(xsel)

            # iteration counter
            iterations = -1

            while sum(flatrecs) < self.mods_goal:
                # Gravity Threshold as mobile target, in order to obtain solutions
                iterations += 1
                # dynamic target adjusted to the standard deviation of the distribution
                gt = static_target - iterations * (gr_std / 10)

                for level in zsel: #range(len(recsel)):
                    for row in ysel: #range(len(recsel[level])):
                        for el in xsel: #range(len(recsel[level][row])):
                            flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
                            # cohefficient is made from a deterministic and a random part proportional to standard deviation
                            mod_cohefficient = self.gr[level][row][el] + (gr_std * random.random()) / 4
                            if mod_cohefficient > gt and sum(flatrecs) * 0.7 < self.mods_goal:
                                recsel[level][row][el] = 1
                                # Updating stop condition
                                flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
                if self.verbose:
                    print("Iteration: ", i, "/", self.iters)
                    print("desired modules over possible: ", self.mods_goal / self.mods_poss)
                    print("Dynamic Threeshold: ", gt)
                    print("before rules: ", sum(flatrecs))
                # Compact building rule
                if self.CompactBuilding:
                    recsel = self.compact_building(recsel)
                    numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                    if self.verbose:
                        print("after compact building:", numberselected, "/", self.mods_goal)
                # Applying exceptions
                if len(self.exceptions) > 0:
                    recsel = self.get_exceptions(recsel, self.exceptions)
                if self.verbose:
                    print("after exceptions:", sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]))
                # Cantilever rule
                if self.Cantilever > 0:
                    recsel = self.cantilever(recsel)
                    numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                    if self.verbose:
                        print("after cantilever:", numberselected, "/", self.mods_goal)
                # Hard Room depth rule
                if self.RoomDepth == "hard":
                    recsel = self.hardroomdepth(recsel)
                    numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                    if self.verbose:
                        print("after hard room depth:", numberselected, "/", self.mods_goal)
                # Soft Room depth rule
                if self.RoomDepth == "soft":
                    recsel = self.softroomdepth(recsel)
                    numberselected = sum([item for sublist in [item for sublist in recsel for item in sublist] for item in sublist])
                    if self.verbose:
                        print("after soft room depth:", numberselected, "/", self.mods_goal)
                # Updating stop condition
                flatrecs = [item for sublist in [item for sublist in recsel for item in sublist] for item in sublist]
                area = sum(flatrecs) * self.MW * self.ML
                if self.verbose:
                    print("_____________________")
            self.combinations.append(recsel)

            # Exporting centers of rectangles
            # centers = get_center(recsel, ML, MW, MH)
            # flatcenters = [item for sublist in [item for sublist in centers for item in sublist] for item in sublist]
            # filename = self.folder + experiment + '_' + str(i) + '.csv'
            # with open(filename, mode = 'w', newline = '') as csv_file:
            #     writer = csv.writer(csv_file, delimiter = ",", quoting = csv.QUOTE_MINIMAL)
            #     for row in flatcenters:
            #         writer.writerow(row)

        # Report experiment specifications:
        repfile = self.folder + "_report.csv"
        with open(repfile, 'r') as f:
            reader = csv.reader(f, delimiter = ';')
            report = list(reader)

        today = datetime.date.today().strftime("%d/%m/%y")
        report.append([today, self.experiment, self.iters, self.shape, self.GForce, self.Weights, self.Cantilever, self.CompactBuilding, self.RoomDepth])
        with open(repfile, 'w', newline = '') as f:
            writer = csv.writer(f, delimiter = ";")
            writer.writerows(report)


        return(self.combinations)
    
    def get_clusters(self):
        self.clusters = []
        
        self.random_configuration()
        
        for comb in self.combinations:
            c = self.get_center(comb)
            clusters_in_conf = []
            for level in range(len(c)):
                centers_in_lev = [item for sublist in c[level] for item in sublist]
                while len(centers_in_lev) > 0:
                    newcluster = [centers_in_lev[0]]
                    candidates = [centers_in_lev[0]]
                    centers_in_lev.pop(0)
                    while len(candidates) > 0:
                        candidate = candidates[0]
                        candidates.pop(0)
                        cX = candidate[0]
                        cY = candidate[1]
                        for totest in centers_in_lev:
                            testX = totest[0]
                            testY = totest[1]
                            cond1 = (cX - testX == 0) and (abs(cY - testY) == self.MW)
                            cond2 = (abs(cX - testX) == self.ML) and (cY - testY == 0)
                            if (cond1 or cond2):
                                newcluster.append(totest)
                                candidates.append(totest)
                                centers_in_lev.remove(totest)
                    clusters_in_conf.append(newcluster)

                    # Export each cluster in a separate csv
                    # clusterfile = "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/cluster_"
                    # for cl in range(len(clusters_in_conf)):
                    #     cl0 = clusters_in_conf[cl]
                    #     file = clusterfile + str(cl) + ".csv"
                    #     with open(file, 'w', newline = '') as f:
                    #         writer = csv.writer(f, delimiter = ",")
                    #         writer.writerows(cl0)
            self.clusters.append(clusters_in_conf)
        return(self.clusters)
    
    def find_gaps(self):
        
        import numpy as np
        
        # INPUT 
        # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
        # clusters:       result from clusters function with this structure [combination [cluster [point[X, Y, Z]]]]
        # OUTPUT 
        self.gaps = [] # List of combinations of clusters  with this structure [combination, [cluster [gap [point [X, Y, Z]]]]]
        
        try:
            self.clusters
        except:
            self.clusters = self.get_clusters()
        for combination in self.clusters:
            gaps_in_combination = []
            for clus in combination:
                # 1: getting bounding box for the cluster
                MaxX = -9e20
                MinX = +9e20
                MaxY = -9e20
                MinY = +9e20
                Z = clus[0][2]
                for point in clus:
                    if point[0] > MaxX:
                        MaxX = point[0]
                    if point[0] < MinX:
                        MinX = point[0]
                    if point[1] > MaxY:
                        MaxY = point[1]
                    if point[1] < MinY:
                        MinY = point[1]
                # 2: getting all the module centers not in the cluster list
                candidates = [[x, y, Z] for x in np.arange(MinX, MaxX, self.ML) for y in np.arange(MinY, MaxY, self.MW)]
                gap_centers = [c for c in candidates if c not in clus]
                # 3 cluster the module centers - based on clusters function
                gaps_in_cluster = []
                while len(gap_centers) > 0:
                    newcluster = [gap_centers[0]]
                    candidates = [gap_centers[0]]
                    gap_centers.pop(0)
                    while len(candidates) > 0:
                        candidate = candidates[0]
                        candidates.pop(0)
                        cX = candidate[0]
                        cY = candidate[1]
                        for totest in gap_centers:
                            testX = totest[0]
                            testY = totest[1]
                            cond1 = (cX - testX == 0) and (abs(cY - testY) == self.MW)
                            cond2 = (abs(cX - testX) == self.ML) and (cY - testY == 0)
                            if (cond1 or cond2):
                                newcluster.append(totest)
                                candidates.append(totest)
                                gap_centers.remove(totest)
                        if len(newcluster) > 0 :
                            gaps_in_cluster.append(newcluster)
                gaps_in_combination.append(gaps_in_cluster)
            self.gaps.append(gaps_in_combination)
        return(self.gaps)
    
    
    def get_perimetral_modules(self):
        
        # OUTPUT 
        self.peri_clus = [] # List of combinations of clusters
        try:
            self.clusters
        except:
            self.clusters = self.get_clusters()
        for combination in self.clusters:
            combination_perimeters = []
            for clus in combination:
                clus_perimeter = []
                for center in clus:
                    # moving the point in the two directions
                    up = [center[0], center[1] + self.MW, center[2]]
                    down = [center[0], center[1] - self.MW, center[2]]
                    right = [center[0] + self.ML, center[1], center[2]]
                    left = [center[0] - self.ML, center[1], center[2]]
                    if not (up in clus and down in clus and right in clus and left in clus):
                        clus_perimeter.append(center)
                combination_perimeters.append(clus_perimeter)
            self.peri_clus.append(combination_perimeters)
        return(self.peri_clus)
    
    def peri_clus_points(self):

        # INPUT 
        # shape:          List with the shape parameters [build. length, build. widht, build. height, mod length, mod width, mod height]
        # perimetral:     result from get_perimetral_modules function

        # OUTPUT 
        peripoints = [] # List corners of modules. Point will be copied on the list just once
        self.get_perimetral_modules()
        self.peri_clus_points = []
        for configuration in self.peri_clus:
            comb_points = []
            for cluster in configuration:
                point_list = []
                for point in cluster:
                    topleft = [point[0] - self.ML / 2, point[1] + self.MW / 2, point[2]]
                    topright = [point[0] + self.ML / 2, point[1] + self.MW / 2, point[2]]
                    bottomright = [point[0] + self.ML / 2, point[1] - self.MW / 2, point[2]]
                    bottomleft = [point[0] - self.ML / 2, point[1] - self.MW / 2, point[2]]
                    for p in [topleft, topright, bottomleft, bottomright]:
                        if p not in point_list:
                            point_list.append(p)
                comb_points.append(point_list)
            self.peri_clus_points.append(comb_points)
    
        return(self.peri_clus_points)
    
    def cluster_perimeter(self, cluster, clusterCenters):
        
        perimeter_list = []  # list of all points in the perimeter
        if self.verbose:
            print("*" * int(1e2))
        remaining_points = cluster[:]
        # all x values within the cluster
        Xs = []
        # all y values within the cluster
        Ys = []
        Z = remaining_points[0][2]
        for point in cluster:
            Xs.append(point[0])
            Ys.append(point[1])
        Xs.sort()
        Ys.sort()
        # List of all the points by Y coordinate
        pointsByY = []
        for Y in Ys:
            to_add = []
            for point in cluster:
                if point[1] == Y:
                    to_add.append(point)
            pointsByY.append(to_add)
        # The starting point will be the one with the smallest X within the ones with the smallest Y
        downX = []
        for point in pointsByY[0]:
            downX.append(point[0])
        minX = min(downX)
        for point in pointsByY[0]:
            if point[0] == minX:
                startingPoint = point
                pointsByY[0].remove(point)
                perimeter_list.append(startingPoint)
                remaining_points.remove(startingPoint)
        # iterative process to get the perimeter chain
        if self.verbose:
            print("points in cluster : ", len(cluster))
            print("starting point: ", startingPoint)
        while perimeter_list[-1] != startingPoint or len(perimeter_list) == 1:
            p0 = perimeter_list[-1]
            up = [p0[0], p0[1] + self.MW, p0[2]]
            down = [p0[0], p0[1] - self.MW, p0[2]]
            right = [p0[0] + self.ML, p0[1], p0[2]]
            left = [p0[0] - self.ML, p0[1], p0[2]]
            if self.verbose:
                print("last point: ", p0)
                print("surrounding point: ", up, down, left, right)
            # check all the points close to p0 within the list of remaining points
            candidates = [value for value in remaining_points if value in [up, down, left, right]]
            if self.verbose:
                print("candidates: ", candidates)
            for can in candidates:
                # if p0 (previous perimeter point) and candidate share the same Y value
                if (can == up or can == down):
                    # condition: in order the point to lay on the perimeter 
                    # there should be a single module center between the two points 
                    # on the right or left
                    possibleCenterR = [((p0[0] + can[0]) / 2) + self.ML / 2, ((p0[1] + can[1]) / 2), Z]
                    possibleCenterL = [((p0[0] + can[0]) / 2) - self.ML / 2, ((p0[1] + can[1]) / 2), Z]
                    # find out if the candidate complies with the perimeter condition 
                    # (separate module centers in the figure from centers not in the figure)
                    # and the direction and way condition
                    if p0[1] < can[1]:
                        cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
                    if p0[1] > can[1]:
                        cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
                    if self.verbose:
                        print("candidate UP/DOWN: ", can)
                        print("point 0: ", p0)
                        print("possible centers : ", possibleCenterR, possibleCenterL)
                        print("cluster centers: ", clusterCenters)
                        print("conditions respected:", cond)
                    if cond:
                        perimeter_list.append(can)
                        remaining_points.remove(can)
                        if self.verbose:
                            print("new Point added", perimeter_list)
                            print("remaining points: ", len(remaining_points), "/", len(cluster))
                            print("-" * int(1e1))
                        break
                # if p0 (previous perimeter point) and candidate share the same X value
                else: 
                    if (can == right or can == left ):
                        # condition: in order the point to lay on the perimeter 
                        # there should be a module center between the two points 
                        # on the top or bottom
                        possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + self.MW / 2, Z]
                        possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - self.MW / 2, Z]
                        # find out if the candidate complies with the perimeter condition 
                        # (separate module centers in the figure from centers not in the figure)
                        # and the direction and way condition
                        if p0[0] > can[0]:
                            cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
                        if p0[0] < can[0]:
                            cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
                        if self.verbose:
                            print("candidate LEFT/RIGHT: ", can)
                            print("point 0: ", p0)
                            print("possible centers : ", possibleCenterT, possibleCenterB)
                            print("cluster centers: ", clusterCenters)
                            print("conditions respected:", cond)
                        if cond:
                            perimeter_list.append(can)
                            remaining_points.remove(can)
                            if self.verbose:
                                print("new Point added", perimeter_list)
                                print("remaining points: ", len(remaining_points), "/", len(cluster))
                                print("-" * int(1e1))
                            break
            # after trying all candidates, if no candidate was added to the 
            # perimeter list, the algorithm verifies the possibility
            # to close the perimeter
            if startingPoint in [up, down, left, right] and p0 == perimeter_list[-1]:
                if self.verbose:
                    print("closing the circle")
                perimeter_list.append(startingPoint)
                break
            # some configurations may lead to vertex overlapping.
            # to solve this, if no other candidate is found, vertices already
            # in the perimeter list can be reconsidered as candidates
            if p0 == perimeter_list[-1]:
                # last point to be excluded to avoid infinte loop
                candidates = [value for value in perimeter_list[:-1] if value in [up, down, left, right]]
                # print("candidates: ", candidates)
                for can in candidates:
                    # if p0 (previous perimeter point) and candidate share the same Y value
                    if (can == up or can == down):
                        possibleCenterR = [((p0[0] + can[0]) / 2) + self.ML / 2, ((p0[1] + can[1]) / 2), Z]
                        possibleCenterL = [((p0[0] + can[0]) / 2) - self.ML / 2, ((p0[1] + can[1]) / 2), Z]
                        if p0[1] < can[1]:
                            cond = possibleCenterR in clusterCenters and possibleCenterL not in clusterCenters
                        if p0[1] > can[1]:
                            cond = possibleCenterR not in clusterCenters and possibleCenterL in clusterCenters
                        if cond:
                            perimeter_list.append(can)
                            if self.verbose:
                                print("old Point added", perimeter_list)
                                print("remaining points: ", len(remaining_points), "/", len(cluster))
                                print("-" * int(1e1))
                            break
                    # if p0 (previous perimeter point) and candidate share the same X value
                    else: 
                        if (can == right or can == left ):
                            possibleCenterT = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) + self.MW / 2, Z]
                            possibleCenterB = [((p0[0] + can[0]) / 2), ((p0[1] + can[1]) / 2) - self.MW / 2, Z]
                            if p0[0] > can[0]:
                                cond = possibleCenterT in clusterCenters and possibleCenterB not in clusterCenters
                            if p0[0] < can[0]:
                                cond = possibleCenterT not in clusterCenters and possibleCenterB in clusterCenters
                            if cond:
                                perimeter_list.append(can)
                                if self.verbose:
                                    print("old Point added", perimeter_list)
                                    print("remaining points: ", len(remaining_points), "/", len(cluster))
                                    print("-" * int(1e1))
                                break
        # removing reduntant starting point
        perimeter_list.pop(-1)
        return(perimeter_list)
    
    def perimeters_gaps(self):
        
        import csv
        
        # OUTPUT 
        self.clus_perimeters = [] # List of combinations of clusters with this structure
                                  # [combination [cluster1_perimeter [point[x, y, z]], [gap_marcator], 
                                  # cluster1_gap1_perimeter[point[X, Y, Z]]]]
        
        perimetral_points = self.peri_clus_points()
        try:
            centers = self.clusters
        except:
            centers = self.get_clusters()
        gaps = self.find_gaps()
        
        for ind_comb in range(len(perimetral_points)):
            configuration = []
            for ind_clus in range(len(perimetral_points[ind_comb])):
                cluster = perimetral_points[ind_comb][ind_clus]
                clus_centers = centers[ind_comb][ind_clus]
                clus_gaps = gaps[ind_comb][ind_clus]
                clus_perimeter = self.cluster_perimeter(cluster, clus_centers)
                if len(clus_gaps) > 0:
                    for gap in clus_gaps:
                        # 1 - get_perimetral_modules - only for this cluster gap
                        gap_perimeter = []
                        for center in gap:
                            # moving the point in the two directions
                            up = [center[0], center[1] + self.MW, center[2]]
                            down = [center[0], center[1] - self.MW, center[2]]
                            right = [center[0] + self.ML, center[1], center[2]]
                            left = [center[0] - self.ML, center[1], center[2]]
                            if not (up in gap and down in gap and right in gap and left in gap):
                                gap_perimeter.append(center)
                        # 2 - perimetral_modules_points - only for this cluster gap
                        gap_point_list = []
                        for point in gap_perimeter:
                            topleft = [point[0] - self.ML / 2, point[1] + self.MW / 2, point[2]]
                            topright = [point[0] + self.ML / 2, point[1] + self.MW / 2, point[2]]
                            bottomright = [point[0] + self.ML / 2, point[1] - self.MW / 2, point[2]]
                            bottomleft = [point[0] - self.ML / 2, point[1] - self.MW / 2, point[2]]
                            for p in [topleft, topright, bottomleft, bottomright]:
                                if p not in gap_point_list:
                                    gap_point_list.append(p)
                        # 3 - Check if gaps are relevant to define the shape or if lay on the perimeter 
                        intersection = [point for point in gap_point_list if point in clus_perimeter]
                        # print("combination: ", ind_comb, "cluster: ", ind_clus, "intersection :", len(intersection))
                        if len(intersection) == 0:
                            # 4 - Find the perimeter of the gap
                            gap_perimeter = self.cluster_perimeter(gap_point_list, gap)
                            # 5 - Aggregate gap to the list of perimeters
                            clus_perimeter.append(["gap", "", ""])
                            for point in gap_perimeter:
                                clus_perimeter.append(point)
                # add perimeter and gaps to the configuration
                clus_perimeter.append(["next cluster", "", ""])
                for element in clus_perimeter:
                    configuration.append(element)
            # add configuration to list of configurations
            self.clus_perimeters.append(configuration)
        
        # Export each point cluster in a separate csv
        perifile = self.folder + self.experiment + "_peri_"
        
        for config in self.clus_perimeters:
            ind = self.clus_perimeters.index(config)
            # periflat = []
            # for peri in config:
            #     for point in peri:
            #         periflat.append(point)
            file = perifile + str(ind) + ".csv"
            with open(file, 'w', newline = '') as f:
                writer = csv.writer(f, delimiter = ",")
                writer.writerows(config)
        return()

In [15]:
## paramsB01 = {
##           "experiment": "test",
##           "folder" : "C:/Users/fsordini/Implenia AG/REPOS - Dokumente/Lokstadt, Rocket-Tigerli/20 Grundlage Machbarkeitsstudie/10 Feasibility/01 Configurations/",
##           "iters": 30, 
##           "shape": [65, 55, 23, 3, 3, 3], 
##           "exceptions": [[30, 0, 0, 15, 100, 6]], 
##           "MaximalArea": 17500 / 0.9,
##           "GForce": 0.1, 
##           "CornerWeights": [-1, 1, 0.4, 0.4], 
##           "corners": False, 
##           "gravityCenters": 1, 
##           "dim": 3, 
##           "CantileverRule": 6, # meters 
##           "CompactBuildingRule": True, 
##           "RoomDepthRule": "soft", 
##           "verbose": False}
## 
## test = GraviationalModel(paramsB01)
## test.perimeters_gaps()

()